In [1]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
import gensim
import numpy as np
import pandas as pd
import tensorflow as tf

from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import optimizers

c:\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [4]:
import logging
import pprint

# for logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Load evaluation dataset of analogy task 
model.accuracy('questions-words.txt')
# execute analogy task like king - man + woman = queen
pprint.pprint(model.most_similar(positive=['woman', 'king'], negative=['man']))

2018-04-23 09:11:33,266 : INFO : precomputing L2-norms of word weight vectors
2018-04-23 09:11:44,244 : INFO : capital-common-countries: 83.6% (423/506)
2018-04-23 09:11:49,974 : INFO : capital-world: 82.7% (1144/1383)
2018-04-23 09:11:50,514 : INFO : currency: 39.8% (51/128)
2018-04-23 09:12:00,094 : INFO : city-in-state: 74.6% (1739/2330)
2018-04-23 09:12:01,469 : INFO : family: 90.1% (308/342)
2018-04-23 09:12:04,732 : INFO : gram1-adjective-to-adverb: 32.3% (262/812)
2018-04-23 09:12:06,281 : INFO : gram2-opposite: 50.5% (192/380)
2018-04-23 09:12:11,717 : INFO : gram3-comparative: 91.9% (1224/1332)
2018-04-23 09:12:14,999 : INFO : gram4-superlative: 88.0% (618/702)
2018-04-23 09:12:18,403 : INFO : gram5-present-participle: 79.8% (694/870)
2018-04-23 09:12:23,209 : INFO : gram6-nationality-adjective: 97.1% (1193/1229)
2018-04-23 09:12:29,050 : INFO : gram7-past-tense: 66.5% (986/1482)
2018-04-23 09:12:33,057 : INFO : gram8-plural: 85.6% (849/992)
2018-04-23 09:12:35,894 : INFO : gr

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]


In [9]:
dim = len(model["my"])
print ("The dimension of mebdding is ", dim)

The dimension of mebdding is  300


In [5]:
model.most_similar(positive=['fruits', "vegetables"], negative=['apple'])

[('fruits_vegetables', 0.6758536100387573),
 ('vegetables_fruits', 0.6009543538093567),
 ('fresh_fruits', 0.593521773815155),
 ('veggies', 0.5848103761672974),
 ('lettuce_tomatoes_cauliflower', 0.5530379414558411),
 ('fresh_fruits_vegetables', 0.5467322468757629),
 ('Vegetables', 0.5445762276649475),
 ('Canned_fruits', 0.5021904110908508),
 ('nutritious_foods', 0.5009387731552124),
 ('vegatables', 0.5002954006195068)]

## Taking the avg of words per document

In [6]:
def review_avg_vec(review):
    avg = np.zeros(dim)
    no_tokens = 0
    for token in review.split():
        try:
            avg = np.array(model[token]) + avg
            
            #print(token)
            no_tokens += 1
        except:
            continue
    #print("Number of token is ", no_tokens)
        
    return avg/no_tokens    
            

In [7]:
with open("C:/Users/gaali/OneDrive/Documents/Courses in RIT/Spring_2018/Capston_Colloquium/Codes/Codes/review_processed_txt.txt", "r") as fh:
    data = fh.read().strip()
    #data = re.sub(r"[^\w\s)]", "", data)
fh.close()

lines = data.split("\n")
paired = [item.split("--->") for item in lines] # review paired with ots label

In [10]:
continuos_data = []

for item in paired:
    continuos_data.append(review_avg_vec(item[0]))
    
#review_avg_vec("I see that you're going to dhddh")


### Building our new dataset

In [11]:
continuos_data = np.array(continuos_data)

continuos_data[0]

array([ 0.00386602,  0.04166776,  0.03788064,  0.11254051, -0.07940813,
        0.03193942,  0.00686593, -0.04304088,  0.05834961,  0.07189473,
       -0.05153309, -0.15895844, -0.00197601, -0.0086122 , -0.06042502,
        0.06951245,  0.05636181,  0.07919242,  0.04242429, -0.0609384 ,
        0.01194694,  0.04038108,  0.0894276 , -0.04918913,  0.08242412,
       -0.01062705, -0.10878129,  0.06221884,  0.02961506, -0.01586845,
       -0.00174209,  0.05397779, -0.0482847 ,  0.01691359,  0.03093373,
        0.02199242,  0.0134364 , -0.00942958,  0.04712885,  0.07128768,
        0.09772283, -0.0578752 ,  0.09312578, -0.02139768,  0.03470351,
        0.0163843 ,  0.01343328,  0.03939264,  0.01867121, -0.0229187 ,
        0.01671115,  0.05122722,  0.00187544,  0.01774077, -0.02351033,
        0.0417425 , -0.01067526, -0.05984775,  0.04191728, -0.03060774,
       -0.03622471,  0.10491926, -0.0829499 , -0.10517883, -0.02067497,
       -0.04734039, -0.02002092,  0.07056072, -0.10847612,  0.09

In [12]:
my_labels = ['BadFood', 'Cost',  'Filthy',  'MissingFood', 'OrderProblem', 'RudeService', 'ScaryMcDs', 'SlowService']

In [13]:
def one_hot_vec(multi_plus_sig):
    this_vec = [0]*len(my_labels)
    splitted = multi_plus_sig.split("+")
    for i in range(len(my_labels)):
        if my_labels[i] in splitted:
            this_vec[i] = 1
    return this_vec

binary_labels = []
for item in paired:
    binary_labels.append(one_hot_vec(item[1]))

binary_labels = np.array(binary_labels)
#binary_respose_df = pd.DataFrame(np.array(binary_labels), columns = my_labels)
binary_respose_df = pd.DataFrame(binary_labels, columns = my_labels)
binary_respose_df.head(n=5)  

,BadFood,Cost,Filthy,MissingFood,OrderProblem,RudeService,ScaryMcDs,SlowService
0,0,0,1,0,1,1,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,1
3,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,1


In [14]:
n_observation = len(continuos_data)

### Midifying our batch fucntion

In [15]:


def batch_giver(x_data, y_data, i, batch_size):
    """
    x_data: All the feature values
    y_data: All the outcome values
    datapoint_size: Number of points/entries in all_xs/all_ys
    batch_size: Configure this to:
             1: stochastic mode
             integer < datapoint_size: mini-batch mode
             datapoint_size: batch mode
    i: Current iteration
    """
    assert len(x_data) == len(y_data)
    datapoint_size = len(x_data)
    if datapoint_size == batch_size:
        # Batch mode so select all points starting from index 0
        batch_start_idx = 0
    
    elif datapoint_size < batch_size:
        # Not possible
        raise ValueError("datapoint_size: %d, must be greater than batch_size: %d" % (datapoint_size, batch_size))
    else:
        # stochastic/mini-batch mode: Select datapoints in batches  
        #from all possible datapoints
        batch_start_idx = i
        batch_end_idx = batch_start_idx + batch_size
        batch_xs = x_data[batch_start_idx:batch_end_idx]
        batch_ys = y_data[batch_start_idx:batch_end_idx]

    # Get batched datapoints into xs, ys, which is fed into
    # 'train_step'
    xs = np.array(batch_xs)
    ys = np.array(batch_ys)
    return xs, ys

In [17]:
batch_giver(continuos_data, binary_labels, 3, 10)

(array([[ 0.01198028,  0.02649311,  0.03051734, ..., -0.05022652,
          0.00959704, -0.04988867],
        [ 0.03331535, -0.04545698,  0.08035131, ...,  0.00061134,
          0.01253583, -0.01928883],
        [ 0.04534725,  0.02352845,  0.03713789, ..., -0.02869215,
          0.0458637 , -0.04795256],
        ...,
        [ 0.0301793 ,  0.01631754,  0.05258077, ..., -0.03502937,
          0.00574514, -0.03385241],
        [ 0.02324023,  0.0160586 ,  0.05701828, ..., -0.0140714 ,
          0.02280412, -0.02682781],
        [ 0.02401664,  0.05149278,  0.00697231, ..., -0.01069364,
          0.06507111, -0.00806635]]), array([[0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 1, 0, 1]]))

## TensorFlow Deep Learning Model

In [18]:
#max_words = len(continuos_data[0])
print("The dimesion is ", dim)
batch_size = 10
epochs = 100
training_portion = int(round(.8*n_observation))
x_train = continuos_data[:training_portion]
y_train = binary_labels[:training_portion]
                       
x_test = continuos_data[training_portion:]
y_test = binary_labels[training_portion:]

print(len(x_train), 'train reviews')
print(len(x_test), 'test reviews')

num_classes = len(y_test[0])
print("The number of classes : ", num_classes, 'classes')
print ("The dimension of dataset is ", continuos_data.shape)
print ("The dimension of target data is ", binary_labels.shape)

print ("The dimension of train data is ", x_train.shape)
print ("The dimension of test data is ", x_test.shape)



The dimesion is  300
941 train reviews
235 test reviews
The number of classes :  8 classes
The dimension of dataset is  (1176, 300)
The dimension of target data is  (1176, 8)
The dimension of train data is  (941, 300)
The dimension of test data is  (235, 300)


### Construction Phase

In [19]:
reset_graph()

n_inputs = dim
n_hidden = 180  # codings
n_hidden2 = 70
n_outputs = num_classes

learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
Y = tf.placeholder(tf.float32, shape=[None, n_outputs])

hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.sigmoid)

#hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.sigmoid)

outputs = tf.layers.dense(hidden1, n_outputs, activation=tf.nn.sigmoid)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - Y))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [20]:
n_iterations = n_observation // batch_size
codings = hidden1

"""
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: x_train, Y: y_train})
    codings_val = codings.eval(feed_dict={X: x_test})
"""

'\nwith tf.Session() as sess:\n    init.run()\n    for iteration in range(n_iterations):\n        training_op.run(feed_dict={X: x_train, Y: y_train})\n    codings_val = codings.eval(feed_dict={X: x_test})\n'

In [21]:
with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
        for iteration in range(n_iterations):
            X_batch, y_batch = batch_giver(continuos_data, binary_labels, iteration, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, Y: y_batch})
        #acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        #print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
        loss_train = reconstruction_loss.eval(feed_dict={X: X_batch, Y:y_batch})   # not shown
        print("\r{}".format(epoch), "Train MSE:", loss_train)           # not shown
    save_path = saver.save(sess, "./my_MLP_model_final.ckpt")

0 Train MSE: 0.13091078
1 Train MSE: 0.12651744
2 Train MSE: 0.11155055
3 Train MSE: 0.101761244
4 Train MSE: 0.09026696
5 Train MSE: 0.082748674
6 Train MSE: 0.073993705
7 Train MSE: 0.06753754
8 Train MSE: 0.065377906
9 Train MSE: 0.06541151
10 Train MSE: 0.07191457
11 Train MSE: 0.06606921
12 Train MSE: 0.060857426
13 Train MSE: 0.049184687
14 Train MSE: 0.047508955
15 Train MSE: 0.05118417
16 Train MSE: 0.054983724
17 Train MSE: 0.055539973
18 Train MSE: 0.057658147
19 Train MSE: 0.056603022
20 Train MSE: 0.056438886
21 Train MSE: 0.04370924
22 Train MSE: 0.034664117
23 Train MSE: 0.032250416
24 Train MSE: 0.030577093
25 Train MSE: 0.035258316
26 Train MSE: 0.029232308
27 Train MSE: 0.028535962
28 Train MSE: 0.030979723
29 Train MSE: 0.0285838
30 Train MSE: 0.027185883
31 Train MSE: 0.026422292
32 Train MSE: 0.0259733
33 Train MSE: 0.025340933
34 Train MSE: 0.025296774
35 Train MSE: 0.025442898
36 Train MSE: 0.025533339
37 Train MSE: 0.025546432
38 Train MSE: 0.025260676
39 Train M

## using the trained net

In [22]:
with tf.Session() as sess:
    saver.restore(sess, "./my_MLP_model_final.ckpt")
    X_new_scaled = [continuos_data[0]] # some new data (scaled from 0 to 1)
    Z = outputs.eval(feed_dict={X: X_new_scaled})
    print("The output is ", Z)

INFO:tensorflow:Restoring parameters from ./my_MLP_model_final.ckpt


2018-04-23 09:16:50,873 : INFO : Restoring parameters from ./my_MLP_model_final.ckpt


The output is  [[6.8456621e-04 1.3170407e-05 9.9863487e-01 1.8204573e-04 9.8830080e-01
  9.9997687e-01 1.4088127e-05 1.4267517e-04]]


### Defining threshold on each component

In [23]:
def continuous_to_binary(vals, threshold):
    """turns a list of continuous values to binary values"""
    res = [None]*len(vals)
    for i in range(len(vals)):
        if vals[i] < threshold:
            res[i] = 0
        else:
            res[i] = 1
    return np.array(res)


In [24]:
def hamming_distance(ls1, ls2):
    n1 = len(ls1)
    n2 = len(ls2)
    if n1 == n2: 
        dist = 0
        for i in range(len(ls1)):
            if ls1[i] != ls2[i]:
                dist+=1
        return dist
    else:
        print("The lists do not have the same length")
        return 

def hamminLoss(ground_truth, predicted):
    sum = 0
    for i in range(len(predicted)): 
        sum+= hamming_distance(np.array(ground_truth[i]), predicted[i])/len(my_labels)
    return sum/len(predicted)


In [25]:
with tf.Session() as sess:
    saver.restore(sess, "./my_MLP_model_final.ckpt") 
    #xs, ys = batch_giver(continuos_data, binary_labels, 4, batch_size)
    xs = continuos_data[950:]
    ys = binary_labels[950:]
    predicts = []
    for i in range(len(xs)):
        X_new_scaled = np.array([xs[i]]) # some new data (scaled from 0 to 1)
        Z = outputs.eval(feed_dict={X: X_new_scaled})
        predicted = continuous_to_binary(Z[0], .5)
        predicts.append(predicted)
        #print(hamming_distance(ys[i], predicted))
        
print("The hamming loss is ", hamminLoss(predicts, ys))
    


INFO:tensorflow:Restoring parameters from ./my_MLP_model_final.ckpt


2018-04-23 09:17:28,455 : INFO : Restoring parameters from ./my_MLP_model_final.ckpt


The hamming loss is  0.17588495575221239


## Using LSTM

### Concatanation!!! 
Wee need to provide the sequential data

In [28]:
embed_dim = dim ## our embedding model dimension


def word_vec_sequence(embed_dim, step, review_ls):
    """
    param embed_dim: the embedding dimension
    param step: number of steps in LSTM
    param review_ls: the list of reviews
    return param res: a list of sequential words 
    """
    res = [] # return value
    for review in review_ls:
        this_review_seq = []
        for j in range(step):
            if j < len(review):
                token = review[j]
                try:
                    this_review_seq.append(model[token])
                except:
                    this_review_seq.append(np.zeros(embed_dim))
            else:
                this_review_seq.append(np.zeros(embed_dim))
        res.append(this_review_seq)
    return res

#word_vec_sequence(embed_dim = 300, step = 4, review_ls = [["hi", "Man"], ["King", "Man"]])           

def text_to_seq(embed_dim, step, ls_of_review):
    """"
    param ls_of_review : list of raw texts
    param embed_dim : emebdding dimension
    """
    res = []
    for item in ls_of_review:
        tokens = item.split(" ")
        res.append(tokens)
    all_seq = word_vec_sequence(embed_dim, step, res) 
    return all_seq

#res = text_to_seq(300, 2, ["I am", "You are"])
ls_reviews = [item[0] for item in paired]

seqential_data = text_to_seq(embed_dim, n_steps, ls_reviews)

#### The first sequential data

In [29]:
seqential_data[0]

[array([-0.03662109,  0.01452637,  0.03515625,  0.23046875, -0.20800781,
         0.26171875, -0.13183594, -0.08740234,  0.07519531,  0.03881836,
        -0.19726562, -0.37109375, -0.22460938, -0.05029297,  0.14648438,
         0.08398438, -0.0625    ,  0.3828125 ,  0.05664062, -0.09277344,
        -0.20898438,  0.11035156,  0.36132812,  0.28710938, -0.15332031,
        -0.16113281, -0.3828125 , -0.05395508, -0.140625  , -0.29101562,
         0.18261719,  0.09326172, -0.19628906, -0.00500488, -0.07910156,
         0.296875  , -0.38085938,  0.44335938,  0.3671875 ,  0.20117188,
         0.07568359, -0.25585938,  0.1953125 ,  0.10253906,  0.23730469,
         0.00772095,  0.1875    , -0.20117188, -0.09277344,  0.10107422,
         0.0246582 ,  0.18457031,  0.19824219,  0.19140625, -0.05419922,
         0.13476562,  0.00506592,  0.10644531, -0.05322266, -0.18945312,
        -0.10498047, -0.01611328, -0.26171875,  0.05004883, -0.04882812,
        -0.3046875 , -0.00799561, -0.14257812, -0.3

In [32]:
reset_graph()

n_inputs = dim
n_neurons = 150
n_steps = 60
n_outputs = num_classes

learning_rate = 0.01

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
Y = tf.placeholder(tf.float32, shape=[None, n_outputs])

lstm_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=True) 
outputs, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states[0], n_outputs)

reconstruction_loss = tf.reduce_mean(tf.square(logits - Y))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [33]:
n_iterations = n_observation // batch_size

with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
        for iteration in range(n_iterations):
            X_batch, y_batch = batch_giver(seqential_data, binary_labels, iteration, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, Y: y_batch})
        #acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        #print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
        loss_train = reconstruction_loss.eval(feed_dict={X: X_batch, Y:y_batch})   # not shown
        print("\r{}".format(epoch), "Train MSE:", loss_train)           # not shown
    save_path = saver.save(sess, "./my_LSTM_60_model_final.ckpt")

0 Train MSE: 0.12173144
1 Train MSE: 0.08748218
2 Train MSE: 0.064165935
3 Train MSE: 0.071245156
4 Train MSE: 0.038619954
5 Train MSE: 0.05082029
6 Train MSE: 0.029365009
7 Train MSE: 0.011689907
8 Train MSE: 0.014240732
9 Train MSE: 0.01745155
10 Train MSE: 0.014877668
11 Train MSE: 0.01600724
12 Train MSE: 0.018596163
13 Train MSE: 0.014797563
14 Train MSE: 0.012190576
15 Train MSE: 0.009823814
16 Train MSE: 0.015865494
17 Train MSE: 0.011719196
18 Train MSE: 0.008499645
19 Train MSE: 0.010168405
20 Train MSE: 0.008157557
21 Train MSE: 0.008423777
22 Train MSE: 0.0058455053
23 Train MSE: 0.004951383
24 Train MSE: 0.008317515
25 Train MSE: 0.0053444826
26 Train MSE: 0.004597717
27 Train MSE: 0.0048103677
28 Train MSE: 0.003200116
29 Train MSE: 0.0027736684
30 Train MSE: 0.0022999917
31 Train MSE: 0.0020328122
32 Train MSE: 0.0013407315
33 Train MSE: 0.0018017644
34 Train MSE: 0.001476637
35 Train MSE: 0.0019240096
36 Train MSE: 0.0014553296
37 Train MSE: 0.001209616
38 Train MSE: 0.0

### Using our trained LSTM

In [35]:
with tf.Session() as sess:
    saver.restore(sess, "./my_LSTM_60_model_final.ckpt") 
    #xs, ys = batch_giver(continuos_data, binary_labels, 4, batch_size)
    xs = seqential_data[950:]
    ys = binary_labels[950:]
    predicts = []
    for i in range(len(xs)):
        X_new_scaled = np.array([xs[i]]) # some new data (scaled from 0 to 1)
        Z = logits.eval(feed_dict={X: X_new_scaled})
        predicted = continuous_to_binary(Z[0], .5)
        predicts.append(predicted)
        #print(hamming_distance(ys[i], predicted))
        
print("The hamming loss is ", hamminLoss(predicts, ys))
    

INFO:tensorflow:Restoring parameters from ./my_LSTM_60_model_final.ckpt


2018-04-23 09:45:23,002 : INFO : Restoring parameters from ./my_LSTM_60_model_final.ckpt


The hamming loss is  0.23506637168141592


## Keras Deep Learning

### Building te model

In [21]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()


model.add(Dense(580, input_shape=(dim,)))
model.add(Dense(400, input_shape=(580,)))
model.add(Dense(280, input_shape=(400,)))
model.add(Dense(150, input_shape=(280,)))
model.add(Dense(95, input_shape=(150,)))
model.add(Dense(50, input_shape=(95,)))
model.add(Dense(35, input_shape=(50,)))
model.add(Dense(15, input_shape=(35,)))


model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes))
model.add(Activation('softmax'))



sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, 
             metrics=['accuracy'])
'''
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
'''


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

x_train shape: (941, 300)
x_test shape: (235, 300)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (941, 8)
y_test shape: (235, 8)
Building model...
Train on 846 samples, validate on 95 samples
Epoch 1/60
846/846 [==============================] - 2s 2ms/step - loss: 0.1562 - acc: 0.1879 - val_loss: 0.1536 - val_acc: 0.1895
Epoch 2/60
846/846 [==============================] - 1s 1ms/step - loss: 0.1528 - acc: 0.2021 - val_loss: 0.1523 - val_acc: 0.1895
Epoch 3/60
846/846 [==============================] - 1s 678us/step - loss: 0.1525 - acc: 0.2104 - val_loss: 0.1522 - val_acc: 0.1895
Epoch 4/60
846/846 [==============================] - 0s 574us/step - loss: 0.1517 - acc: 0.2128 - val_loss: 0.1515 - val_acc: 0.1895
Epoch 5/60
846/846 [==============================] - 1s 601us/step - loss: 0.1501 - acc: 0.2187 - val_loss: 0.1513 - val_acc: 0.1895
Epoch 6/60
846/846 [==============================] - 1s 608us/step - loss: 0.1504 - acc:

235/235 [==============================] - 0s 188us/step
Test score: 0.11822236503692383
Test accuracy: 0.41702128439507585
